- [ ] sheet 2 intl departure needs to be taken into account 
- [ ] eng arrival and base departure needs to be finalised
- [ ] move entire code into seperate repositiory 
- [ ] understand stremlit 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pickle
import re

temp_list = pickle.load(open("gsheet_pickle.p", "rb"))
alpha = temp_list[0]
master_df = pd.DataFrame(columns =['Dept Arp', 'engg_arr', 'engg_dep', 'deptype', 'Arvl Arp', 'Flt Desg',
       'Block Time', 'D2', 'A2', 'Text Comment'])




In [13]:
depDay = "D4"
arrDay = "A4"

x_df =alpha.loc[(alpha["Text Comment"]=="A10") & ((alpha["variable"]==depDay) | (alpha["variable"]==arrDay) )]
# removing false entries and sorting as false entries cannot be sorted 

def df_flat_function(test_df , dep_D,arr_D):
    test_df = test_df.sort_values(by="value")
    test_df =test_df.pivot(index = ['Dept Arp', 'engg_arr', 'deptype', 'Arvl Arp', 'Flt Desg','Text Comment', 'Block Time'] ,columns = "variable" ,values = 'value')
    test_df = test_df.sort_values(by=dep_D)
    flattened = pd.DataFrame(test_df.to_records())
    flattened["engg_dep"]=False
    flattened =  flattened[['Dept Arp', 'engg_arr','engg_dep' ,'deptype', 'Arvl Arp', 'Flt Desg','Block Time', dep_D, arr_D,'Text Comment']]
    return flattened

right_df = df_flat_function(x_df ,depDay,arrDay)
right_df

,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,D4,A4,Text Comment
0,IXC,False,False,False,PNQ,6E 6514,2:25,2021-02-04 07:35:00,2021-02-04 10:00:00,A10
1,PNQ,False,False,False,IXC,6E 6515,2:30,2021-02-04 10:30:00,2021-02-04 13:00:00,A10
2,IXC,False,False,False,GOI,6E 0389,2:45,2021-02-04 14:25:00,2021-02-04 17:10:00,A10
3,GOI,False,False,False,IXC,6E 0391,2:40,2021-02-04 17:40:00,2021-02-04 20:20:00,A10


In [3]:
# pseudo code 
# sort as per column "D2 "
# last row copy and add a new row for base arrival case 
# first row  make engg equal to D2 
# all middle rows make engg arrival  previous row A2 
# add a day column at the last reprenting the day 

In [14]:

def make_last_row(onedf ,arr_D):
    temp_false_list = []
    for india in range(flattened.shape[1]):
        temp_false_list.append(False)
    temp_false_list
    onedf.loc[len(onedf)] = temp_false_list
    onedf.loc[(len(onedf)-1), "Dept Arp" ] =onedf.loc[(len(onedf)-2), "Arvl Arp" ]
    onedf.loc[(len(onedf)-1), "engg_arr" ] =onedf.loc[(len(onedf)-2), arr_D]
    onedf.loc[(len(onedf)-1), "Text Comment" ] =onedf.loc[(len(onedf)-2), "Text Comment" ]
    return onedf
right_df = make_last_row(right_df,arrDay)
right_df





,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,D4,A4,Text Comment
0,IXC,False,False,False,PNQ,6E 6514,2:25,2021-02-04 07:35:00,2021-02-04 10:00:00,A10
1,PNQ,False,False,False,IXC,6E 6515,2:30,2021-02-04 10:30:00,2021-02-04 13:00:00,A10
2,IXC,False,False,False,GOI,6E 0389,2:45,2021-02-04 14:25:00,2021-02-04 17:10:00,A10
3,GOI,False,False,False,IXC,6E 0391,2:40,2021-02-04 17:40:00,2021-02-04 20:20:00,A10
4,IXC,2021-02-04 20:20:00,False,False,False,False,False,False,False,A10


In [7]:
for xray in range(0,len(onedf)):
    if xray  == 0:
        onedf.loc[xray,"deptype"] = "baseDep"
        onedf.loc[xray,"engg_arr"] = "baseAcft"
        onedf.loc[xray,"engg_dep"] = onedf.loc[xray,"D2"]
    elif xray == len(onedf)-1:
        
        onedf.loc[xray,"deptype"] = "baseArr"
        print("lastroe")
    else:
        onedf.loc[xray,"engg_arr"] = onedf.loc[(xray-1),"A2"]
        onedf.loc[xray,"engg_dep"] = onedf.loc[(xray),"D2"]



onedf.columns


master_df = master_df.append(onedf,ignore_index=True)
master_df


lastroe


,Dept Arp,engg_arr,engg_dep,deptype,Arvl Arp,Flt Desg,Block Time,D2,A2,Text Comment
0,IXC,baseAcft,2021-02-02 07:35:00,baseDep,PNQ,6E 6514,2:25,2021-02-02 07:35:00,2021-02-02 10:00:00,A10
1,PNQ,2021-02-02 10:00:00,2021-02-02 10:30:00,False,IXC,6E 6515,2:30,2021-02-02 10:30:00,2021-02-02 13:00:00,A10
2,IXC,2021-02-02 13:00:00,2021-02-02 14:25:00,False,GOI,6E 0389,2:45,2021-02-02 14:25:00,2021-02-02 17:10:00,A10
3,GOI,2021-02-02 17:10:00,2021-02-02 17:40:00,False,IXC,6E 0391,2:40,2021-02-02 17:40:00,2021-02-02 20:20:00,A10
4,IXC,2021-02-02 20:20:00,False,baseArr,False,False,False,False,False,A10
5,IXC,baseAcft,2021-02-02 07:35:00,baseDep,PNQ,6E 6514,2:25,2021-02-02 07:35:00,2021-02-02 10:00:00,A10
6,PNQ,2021-02-02 10:00:00,2021-02-02 10:30:00,False,IXC,6E 6515,2:30,2021-02-02 10:30:00,2021-02-02 13:00:00,A10
7,IXC,2021-02-02 13:00:00,2021-02-02 14:25:00,False,GOI,6E 0389,2:45,2021-02-02 14:25:00,2021-02-02 17:10:00,A10
8,GOI,2021-02-02 17:10:00,2021-02-02 17:40:00,False,IXC,6E 0391,2:40,2021-02-02 17:40:00,2021-02-02 20:20:00,A10
9,IXC,2021-02-02 20:20:00,False,baseArr,False,False,False,False,False,A10
